### Imports

In [1]:
!pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement apsw==3.33.0.post1
ERROR: No matching distribution found for apsw==3.33.0.post1


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [33]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import copy
import os

from torchvision.datasets import MNIST

from profilehooks import profile
    
import matplotlib.pylab as plt

#### Get number of threads and choose range of number of clusters

In [34]:
N_PROC = 2 #CHANGE THIS
N_TREES = 1000
N_TRAIN_CYCLES = 5

### Workload

Run a random forest classifier on the MNIST dataset. The training process is CPU only with the trees distributed across N_PROC. Note: obviously, we are not claiming a random forest classifier is the most appropriate model for an image classification problem on raw pixel features.

In [35]:
data = MNIST('./', download=True)

In [36]:
data.train_labels

/home/sanjay/venv_general/lib64/python3.8/site-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


tensor([5, 0, 4,  ..., 5, 6, 8])

In [37]:
#flatten each image
data_features = data.train_data.numpy().reshape(data.train_data.shape[0], -1).astype(np.float32)
data_labels = data.train_labels.numpy()
print(data_features.shape)
print(data_labels.shape)

(60000, 784)
(60000,)


/home/sanjay/venv_general/lib64/python3.8/site-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [38]:
model = RandomForestClassifier(n_estimators = N_TREES,
                               max_depth =15,
                               n_jobs = N_PROC,
                               oob_score = True)

### Some run times

In [42]:
@profile(immediate=True)
def train(data_features, data_labels, N_TRAIN_CYCLES):
    scores = []
    
    for _ in range(N_TRAIN_CYCLES):
        model = RandomForestClassifier(n_estimators = N_TREES,
                               max_depth =15,
                               n_jobs = N_PROC,
                               oob_score = True)
        
        model.fit(data_features, data_labels)
        
        scores.append(model.oob_score_)
    
    return scores

In [ ]:
train(data_features, data_labels, 1)

In [ ]:
plt.hist(scores)